In [1]:
from functions import *
from random import sample
from htmldate import find_date
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup
import json
from langid.langid import LanguageIdentifier, model
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.ticker as ticker
import matplotlib.cm as cm
import matplotlib as mpl

%matplotlib inline

In [2]:
base_path = "/media/ruben/Data Drive/react-data/npg"
#base_path = "D:/react-data/protest/carlo-batch"

In [3]:
# Get Publication Dates

# dates_ref = dict()

# for photo in [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]:
#     photo_folder = os.path.join(base_path, photo)
#     with open(os.path.join(photo_folder,"dates.txt"),'r') as f:
#         x = f.readlines()
#     dates_ref.update({d.split('|')[0]:d.split('|')[-1].replace('\n','') for d in x if d.split('|')[-1].replace('\n','') != "na" and "ERROR" not in d.split('|')[-1].replace('\n','')})

# Get URL-ID from entity files
ids_ = dict()

for photo in [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]:
    photo_folder = os.path.join(base_path, photo)
    
    folders = [fol for fol in os.listdir(photo_folder) if os.path.isdir(os.path.join(photo_folder,fol)) and "source" not in fol and "context" not in fol]
    
    for f in folders:
        fn = os.path.join(photo_folder, f, "txt","parsed_text.json")

        with open(fn) as fp:
            file = json.load(fp)
        
        for id_ in file.keys():
            
            identif = id_.split("html_")[0].split('html/')[-1][:-1]
            url = id_.split("html_")[1]
            ids_.update({url:identif})

In [5]:
# Open translation file
refdf = pd.read_csv("/media/ruben/FEF44259F44213F5/Users/Ruben/Documents/GitHub/ReACT_GCV/data/images_tables_article_carlo/descriptive-full.csv")
refdf_title = dict(zip(refdf['image'],refdf['pretty_title']))
#refdf_country = dict(zip(refdf['pretty_title'],refdf['country']))

In [4]:
d_ = dict()
for photo in [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]:
    photo_folder = os.path.join(base_path, photo)
    
    with open(os.path.join(photo_folder,'languages-'+photo+".json"),'r') as f:
        lang = json.load(f)
    
    languages = []
    
    for iterkey,items1 in lang.items():
        languages += [v[0] for k,v in items1.items()]
    
    languages = dict(Counter(languages))
    #pretty_title = refdf_title[photo]
    d_.update({photo:languages})

In [5]:
d_ = [[[photo,lan,score]for lan,score in items1.items()] for photo,items1 in d_.items()]
d_ = [item for sublist in d_ for item in sublist]
df = pd.DataFrame(d_,columns=['photo','lan','count'])
df = df.pivot(index='photo',columns='lan',values='count').fillna(0)
#df.to_csv('/media/ruben/FEF44259F44213F5/Users/Ruben/Documents/GitHub/ReACT_GCV/data/images_tables_article/languages-per-photo.csv')

In [6]:
lan_counts = [[c,int(df[c].sum())] for c in list(df.columns)[1:]]
lan_sel = [[l,c] for l,c in lan_counts if c > 20]
other_lan = [l[0] for l in lan_counts if l[0] not in [x[0] for x in lan_sel]]

In [7]:
#df['country'] = [refdf_country[c] for c in df.index]
#dfs = df.groupby('country').sum()#.reset_index()
dfs = df
dfs['other'] = dfs[other_lan].sum(axis=1)
dfs = dfs[[i[0] for i in lan_sel] + ['other']]

In [8]:
dfsn = []

for i in dfs.index:
    t = list(dfs.loc[i])
    tsum = sum(t)
    t = [int(round(x / tsum * 100)) for x in t]
    t = [i] + t
    dfsn.append(t)
dfsn = pd.DataFrame(dfsn)
dfsn.index = dfsn.iloc[:,0]
dfsn = dfsn.iloc[:,1:]
dfsn.columns = dfs.columns
dfsn = dfsn.T

In [11]:
from matplotlib import colors
def b_g(s, cmap='GnBu', low=0, high=0):
    # Pass the columns from Dataframe A 
    a = dfsn.loc[:,s.name].copy()
    rng = a.max() - a.min()
    norm = colors.Normalize(a.min() - (rng * low),
                        a.max() + (rng * high))
    normed = norm(a.values)
    c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    return ['background-color: %s' % color for color in c]
dfsn2 = dfsn.style.apply(b_g,cmap='summer').to_excel("/media/ruben/FEF44259F44213F5/Users/Ruben/Documents/GitHub/ReACT_GCV/data/images_tables_article_carlo/languages-country-colored-c.xlsx", engine="openpyxl")

In [14]:
Counter(dict(lan_counts)).most_common(25)

[('en', 15919),
 ('es', 3176),
 ('it', 2076),
 ('zh', 1768),
 ('pt', 1707),
 ('de', 1202),
 ('fr', 1136),
 ('nl', 987),
 ('an', 737),
 ('cy', 588),
 ('id', 504),
 ('pl', 500),
 ('da', 486),
 ('sq', 485),
 ('tr', 456),
 ('vi', 455),
 ('hr', 449),
 ('ru', 415),
 ('ko', 355),
 ('ar', 331),
 ('no', 289),
 ('ca', 231),
 ('br', 225),
 ('sl', 218),
 ('el', 207)]

In [13]:
type(dfsn2)

NoneType